# 🤗 x 🦾: Training ACT with LeRobot Notebook

Welcome to the **LeRobot ACT training notebook**! This notebook provides a ready-to-run setup for training imitation learning policies using the [🤗 LeRobot](https://github.com/huggingface/lerobot) library.

In this example, we train an `ACT` policy using a dataset hosted on the [Hugging Face Hub](https://huggingface.co/), and optionally track training metrics with [Weights & Biases (wandb)](https://wandb.ai/).

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=YOUR_USERNAME/YOUR_DATASET`)
- Optional: A [wandb](https://wandb.ai/) account if you want to enable training visualization
- Recommended: GPU runtime (e.g., NVIDIA A100) for faster training

## ⏱️ Expected Training Time
Training with the `ACT` policy for 100,000 steps typically takes **about 1.5 hours on an NVIDIA A100** GPU. On less powerful GPUs or CPUs, training may take significantly longer.

## Example Output
Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`. If `wandb` is enabled, progress will also be visualized in your wandb project dashboard.


## Install conda
This cell uses `condacolab` to bootstrap a full Conda environment inside Google Colab.


In [ ]:
from google.colab import userdata

modelname = userdata.get('MODELNAME')

%env MODELNAME=$modelname

!echo $MODELNAME

env: MODELNAME=act_training_v0_1
act_training_v0_1


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:07
🔁 Restarting kernel...


## Install LeRobot
This cell clones the `lerobot` repository from Hugging Face, installs FFmpeg (version 7.1.1), and installs the package in editable mode.


In [ ]:
!git clone https://github.com/huggingface/lerobot.git
!conda install ffmpeg=7.1.1 -c conda-forge
!cd lerobot && pip install -e .

Cloning into 'lerobot'...
remote: Enumerating objects: 40244, done.
remote: Counting objects: 100% (341/341), done.
remote: Compressing objects: 100% (219/219), done.
remote: Total 40244 (delta 251), reused 122 (delta 122), pack-reused 39903 (from 5)
Receiving objects: 100% (40244/40244), 170.02 MiB | 16.23 MiB/s, done.
Resolving deltas: 100% (26221/26221), done.
Filtering content: 100% (45/45), 69.03 MiB | 20.32 MiB/s, done.
Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - ffmpeg=7.1.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    alsa-lib-1.2.14            |       hb9d3cd8_0         553 KB  conda-forge
    aom-3.9.1                  |       hac33072_0         2.6 MB  conda-forge
    attr-2.5.2                 |       h39aace5_0          66 KB  conda-forge

## Weights & Biases login
This cell logs you into Weights & Biases (wandb) to enable experiment tracking and logging.

In [ ]:
#!wandb login

## Conversion du dataset pour entrainement


In [ ]:
# transformation du dataset
!pip install --upgrade huggingface_hub

In [ ]:
from huggingface_hub import login
login()

In [ ]:
!python -m lerobot.datasets.v30.convert_dataset_v21_to_v30 --repo-id=Heuzef/rectangle_v1

Trying to download v3.0 version of the dataset from the hub...
Dataset does not have an uploaded v3.0 version. Continuing with conversion.
Fetching 266 files:   0% 0/266 [00:00<?, ?it/s]
data/chunk-000/episode_000001.parquet:   0% 0.00/16.2k [00:00<?, ?B/s]

data/chunk-000/episode_000005.parquet:   0% 0.00/14.6k [00:00<?, ?B/s]


data/chunk-000/episode_000003.parquet:   0% 0.00/17.2k [00:00<?, ?B/s]



data/chunk-000/episode_000000.parquet:   0% 0.00/16.2k [00:00<?, ?B/s]




data/chunk-000/episode_000004.parquet:   0% 0.00/17.8k [00:00<?, ?B/s]





data/chunk-000/episode_000002.parquet:   0% 0.00/15.9k [00:00<?, ?B/s]






README.md: 4.35kB [00:00, 13.5MB/s]







.gitattributes: 2.46kB [00:00, 10.4MB/s]
Fetching 266 files:   0% 1/266 [00:00<03:25,  1.29it/s]






data/chunk-000/episode_000006.parquet:   0% 0.00/16.4k [00:00<?, ?B/s]







data/chunk-000/episode_000007.parquet:   0% 0.00/15.1k [00:00<?, ?B/s]





data/chunk-000/episode_000002.parquet: 100% 15.9k/15.9k [00:01<00:

## Gestion dossier outputs


In [ ]:
import os
# listing du dossier outputs
!mkdir -p /content/outputs/train
os.chdir("/content/outputs/train/")
!ls

# suppression dossier entrainement si besoin de relancer. Sinon on a une exception
#!rm -r /content/outputs/train/NOM

## Start training ACT with LeRobot

This cell runs the `train.py` script from the `lerobot` library to train a robot control policy.  

Make sure to adjust the following arguments to your setup:

1. `--dataset.repo_id=YOUR_HF_USERNAME/YOUR_DATASET`:  
   Replace this with the Hugging Face Hub repo ID where your dataset is stored, e.g., `pepijn223/il_gym0`.

2. `--policy.type=act`:  
   Specifies the policy configuration to use. `act` refers to [configuration_act.py](../lerobot/common/policies/act/configuration_act.py), which will automatically adapt to your dataset’s setup (e.g., number of motors and cameras).

3. `--output_dir=outputs/train/...`:  
   Directory where training logs and model checkpoints will be saved.

4. `--job_name=...`:  
   A name for this training job, used for logging and Weights & Biases.

5. `--policy.device=cuda`:  
   Use `cuda` if training on an NVIDIA GPU. Use `mps` for Apple Silicon, or `cpu` if no GPU is available.

6. `--wandb.enable=true`:  
   Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

In [ ]:
!cd /content/ && python lerobot/src/lerobot/scripts/lerobot_train.py -h

usage: lerobot_train.py [-h] [--config_path str] [--dataset str]
                        [--dataset.repo_id str] [--dataset.root [str]]
                        [--dataset.episodes [List]] [--image_transforms str]
                        [--dataset.image_transforms.enable bool]
                        [--dataset.image_transforms.max_num_transforms int]
                        [--dataset.image_transforms.random_order bool]
                        [--dataset.image_transforms.tfs Dict]
                        [--dataset.revision [str]]
                        [--dataset.use_imagenet_stats bool]
                        [--dataset.video_backend str]
                        [--dataset.streaming bool] [--env str]
                        [--env.type {aloha,pusht,gym_manipulator,libero,metaworld}]
                        [--env.visualization_width int]
                        [--env.visualization_height int] [--robot str]
                        [--env.robot.type {}] [--teleop str]
             

## Test if gpu is available

In [ ]:
# Vérification de GPU CUDA avec le moins de dépendances possibles

import platform
import sys
import subprocess
import os

print("=== 🔍 INFORMATIONS SYSTÈME ===")
print(f"OS              : {platform.system()} {platform.release()}")
print(f"Version Python  : {sys.version}")
print("-" * 60)

# Bloc 1 : PyTorch (si disponible)
try:
    import torch
    print("=== ⚙️ INFOS CUDA via PyTorch ===")
    print(f"PyTorch version : {torch.__version__}")
    print(f"CUDA disponible : {torch.cuda.is_available()}")
    print(f"CUDA version    : {torch.version.cuda}")
    print(f"CUDNN version   : {torch.backends.cudnn.version() if torch.backends.cudnn.is_available() else 'Non disponible'}")

    if torch.cuda.is_available():
        nb_gpus = torch.cuda.device_count()
        print(f"\nNombre de GPU détectés : {nb_gpus}\n")
        for i in range(nb_gpus):
            props = torch.cuda.get_device_properties(i)
            print(f"--- GPU {i} ---")
            print(f"Nom                : {props.name}")
            print(f"Capacité de calcul : {props.major}.{props.minor}")
            print(f"Mémoire totale     : {props.total_memory / 1024**3:.2f} Go")
            print(f"Processeurs multi  : {props.multi_processor_count}")
            #print(f"Fréquence horloge  : {props.clock_rate / 1e3:.0f} MHz")
            print()
    else:
        print("🚫 Aucun GPU CUDA détecté par PyTorch.")

except ImportError:
    print("❌ PyTorch n'est pas installé (import torch impossible).")

print("-" * 60)

# Bloc 2 : TensorFlow (si dispo)
try:
    import tensorflow as tf
    print("=== 🧠 INFOS GPU via TensorFlow ===")
    print(f"TensorFlow version : {tf.__version__}")
    gpus = tf.config.list_physical_devices('GPU')
    print(f"GPU détecté(s) : {len(gpus)}")
    for g in gpus:
        print(f" - {g}")
except ImportError:
    print("❌ TensorFlow n'est pas installé (import tensorflow impossible).")

print("-" * 60)

# Bloc 3 : NVIDIA-SMI (si disponible)
print("=== 🧩 INFOS NVIDIA-SMI ===")
try:
    result = subprocess.run(["nvidia-smi"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode == 0:
        print(result.stdout)
    else:
        print("⚠️ La commande 'nvidia-smi' a échoué :")
        print(result.stderr)
except FileNotFoundError:
    print("❌ La commande 'nvidia-smi' n'est pas disponible sur ce système.")

=== 🔍 INFORMATIONS SYSTÈME ===
OS              : Linux 6.6.105+
Version Python  : 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
------------------------------------------------------------
=== ⚙️ INFOS CUDA via PyTorch ===
PyTorch version : 2.8.0+cu126
CUDA disponible : True
CUDA version    : 12.6
CUDNN version   : 91002

Nombre de GPU détectés : 1

--- GPU 0 ---
Nom                : NVIDIA A100-SXM4-40GB
Capacité de calcul : 8.0
Mémoire totale     : 39.56 Go
Processeurs multi  : 108

------------------------------------------------------------
=== 🧠 INFOS GPU via TensorFlow ===
TensorFlow version : 2.19.0
GPU détecté(s) : 1
 - PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
------------------------------------------------------------
=== 🧩 INFOS NVIDIA-SMI ===
Sun Oct 19 08:21:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 1

# Entrainement

In [ ]:
!rm -r /content/outputs/train/act_rectangle_v2
!cd /content/ && python lerobot/src/lerobot/scripts/lerobot_train.py \
  --dataset.repo_id=Heuzef/rectangle_v1 \
  --policy.type=act \
  --output_dir=outputs/train/act_rectangle_v2  \
  --job_name=train_act_rectangle_v2 \
  --policy.device=cuda \
  --wandb.enable=false \
  --policy.repo_id=Heuzef/act_rectangle_v2 \
  --batch_size=16 \
  --log_freq=50 \
  --save_freq=2000 \
  --optimizer.lr=1e-5 \
  --step=1000

rm: cannot remove '/content/outputs/train/act_rectangle_v2': No such file or directory
python3: can't open file '/content/lerobot/src/lerobot/scripts/lerobot_train.py': [Errno 2] No such file or directory


In [ ]:
!pwd
!ls act_rectangle_v0_1/checkpoints/last/training_state/

In [ ]:
# Reprise du dernier checkpoint après une interruption :

!cd /content/ && python lerobot/src/lerobot/scripts/lerobot_train.py \
  --dataset.repo_id=Heuzef/rectangle \
  --policy.type=act \
  --output_dir=outputs/train/act_rectangle_v0_1  \
  --job_name=train_act_rectangle_v0_1 \
  --policy.device=cuda \
  --wandb.enable=true \
  --policy.repo_id=Heuzef/act_rectangle_v0_1 \
  --log_freq=20 \
  --save_freq=80 \
  --use_policy_training_preset=true \
  --step=200

In [ ]:
!echo $MODELNAME

In [ ]:
!hf upload $MODELNAME outputs/train/act_rectangle_v0_1/checkpoints/000080/pretrained_model
!cd /content/ && python lerobot/src/lerobot/scripts/lerobot_train.py \
  --config_path=act_rectangle_v0_1 \
  --resume=true

In [ ]:
# Test avec smolvla (optionnel)

!rm -r /content/outputs/train/dataset_rectangle_v0
!cd /content/ && python lerobot/src/lerobot/scripts/lerobot_train.py \
  --dataset.repo_id=Heuzef/rectangle \
  --policy.path=smolvla_base \
  --output_dir=outputs/train/smol_rectangle_v0  \
  --job_name=train_smol_rectangle_v0 \
  --policy.device=cuda \
  --wandb.enable=false \
  --policy.repo_id=Heuzef/smol_rectangle_v0 \
  --step=100

## Login into Hugging Face Hub
Now after training is done login into the Hugging Face hub and upload the last checkpoint

In [ ]:
!huggingface-cli login

In [ ]:
!hf upload Heuzef/model_save \
  /content/outputs/train/act_rectangle_v0_1/checkpoints/last/pretrained_model